# Setup

Turn on autoreload. All modules will be reloaded before excecuting each cell. Even objects that already exist will receive the new method defintions.

In [ ]:
%load_ext autoreload
%autoreload 2

Import stuff.

In [ ]:
import logging
import sys
import os
import atexit

sys.path.insert(0, os.path.join(os.getcwd(), ".."))
from cirrus import automate, GridSearch, LogisticRegression, graph

Configure logging so that debug-level log messages are printed.

In [ ]:
log = logging.getLogger("cirrus")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

# Done on release

Create the server image (and an instance for that purpose).

In [ ]:
instance_for_server_image = automate.Instance(
    "cirrus_instance_for_server_image",
    ami_id="ami-3a674d5a",
    region="us-west-1",
    disk_size=32,
    typ="m5.2xlarge",
    username="ec2-user"
)
instance_for_server_image.start()
automate.make_server_image(
    "cirrus_server_image",
    "s3://cirrus-public/from_shared_vm/executables",
    instance_for_server_image
)
instance_for_server_image.cleanup()

Make the lambda package.

In [ ]:
automate.make_lambda_package(
    "s3://cirrus-public/0/lambda-package",
    "s3://cirrus-public/from_shared_vm/executables"
)

# Done by user

Make the lambda.

In [ ]:
automate.make_lambda("cirrus_worker", "s3://cirrus-public/0/lambda-package", 15)

Make the instance for the parameter servers.

In [ ]:
instance_for_ps = automate.Instance(
    "cirrus_instance_for_ps",
    ami_name="cirrus_server_image",
    region="us-west-1",
    disk_size=32,
    typ="m5.2xlarge",
    username="ec2-user"
)
instance_for_ps.start()

Define the logistic regression configuration.

In [ ]:
basic_params = {
    'n_workers': 2,
    'n_ps': 1,
    'lambda_size': 128,
    'dataset': "shea-criteo-kaggle-19b2",
    'learning_rate': 0.01,
    'epsilon': 0.0001,
    'progress_callback': None,
    'timeout': 60,
    'threshold_loss': 0,
    'opt_method': 'adagrad',
    'minibatch_size': 2,
    'model_bits': 19,
    'train_set': (0,6),
    'test_set': (7,8)
}

Start a grid search.

In [ ]:
gs = GridSearch(
    task=LogisticRegression,
    param_base=basic_params,
    hyper_vars=["learning_rate", "lambda_size"],
    hyper_params=[[0.1, 0.2], [128]],
    instances=[instance_for_ps]
)
gs.set_threads(2)
gs.run(UI=True)

View the progress of the experiment.

In [ ]:
graph.display_dash()

# Utilities

Write the SSH key for the instance to a file.

In [ ]:
with open("key.pem", "w+") as f:
    f.write(instance_for_ps.private_key())

View the instance's public IP address.

In [ ]:
instance_for_ps.public_ip()

View the output of the parameter server of the first experiment.

In [ ]:
gs.cirrus_objs[0].ps.error_output()

In [ ]:
gs.cirrus_objs[1].ps.error_output()

In [ ]:
gs.kill_all()

Create a new SSH connection to the instance. Useful when the connection times out while you're aware from the computer.

In [ ]:
instance_for_ps._connect_ssh()

Do cleanup. Normally this is called on exit, but Jupyter does not reliably allow this to complete.

In [ ]:
atexit._run_exitfuncs()